In [ ]:
%pylab inline

import sys
import os
par_dir = os.path.split(os.getcwd())[0]
if par_dir not in sys.path:
    sys.path.append(par_dir)
# sys.path.append(r'/Users/artemgolovizin/GitHub')
from scipy.optimize import curve_fit
import inspect
import pickle
import imp
import re
import json

from IPython.html import widgets
from IPython.display import display
from IPython.html.widgets import interact, interactive, fixed

from IPython import parallel

import thulium_python_lib.image_processing_new as impr
rc1 = parallel.Client()
lview = rc1.load_balanced_view()
dview = rc1.direct_view()
with dview.sync_imports():
    import sys, os    
dview['par_dir'] = par_dir
%px if par_dir not in sys.path: sys.path.append(par_dir)
#%px if r'/Users/artemgolovizin/GitHub' not in sys.path: sys.path.append(r'/Users/artemgolovizin/GitHub')
%px import thulium_python_lib.image_processing_new as impr
%px import imp
%px from IPython.parallel import bind_kernel; bind_kernel()

In [ ]:
# to reload library on remote and local engine
# %px imp.reload(impr)
# imp.reload(impr)

In [ ]:
def exp_decay(t, N0, tau, background):
    return N0 * exp(- t / tau) + background
def exp_decay_no_bg(t, N0, tau):
    return N0 * exp(- t / tau)
def cloud_expansion(t, T, r0, t0):
    k_b = 1.38e-23
    m = 169 * 1.66e-27
    return sqrt( r0**2 + 2 * k_b * T * (t + t0)**2 / m)
def tow_body_loss(t, N0, betta, background):
    return 1 / ( betta * t + 1 / N0) + background
def exp_plus_tw_body_decay(t, N0, tau, betta,  background):
    return N0 * exp(- t / tau) / ( 1 + betta * N0 * tau * (1 - exp( -t / tau))) + 0 * background
def two_frac_decay(t, N0, N1, tau, betta,  background):
    return exp_decay(t, N0, tau, 0) + exp_plus_tw_body_decay(t,N1, tau, betta,  0) + abs(background)
def two_frac_decay_no_bg(t, N0, N1, tau, betta, background):
    return two_frac_decay(t, N0, N1, tau, betta,0)

In [ ]:
os.chdir(r'D:\!Data\2015_06_18')

In [ ]:
old_ks = dict()

In [ ]:
with open('all_data.txt', 'rb') as handle:
    data = pickle.loads(handle.read())

ks = list(data.keys())
ks.sort()
ch_boxes = dict()
ks_n = []
for k in ks:
    # optional - uncomment line below to sift only keys with specific marker, as 'CL'
#    if 'CL' not in k.upper(): continue
    ch_boxes[k]=old_ks.get(k,True)
    ks_n.append(k)
ks = ks_n
fg = None   
def plot_gr(l_pos,**ar):
    fig,ax = subplots() 
    global fg, old_ks
    old_ks = dict()
    for k in ks:
        old_ks[k]=ar[k]
        if ar[k]:
            val=datat[k]
            x,y=impr.data2_sort(val[0],val[1])
            # here it's time to modify data
#             x = mod_from_AOM_to_real_freq(x, k, 420)
            #x,y = x[1:],y[1:]
            ax.plot(x,y,'-*',label=k)
    legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=15)
    #xscale('log')
    #xlabel('Current, A')
    #xlabel('AOM frequency, MHz')
    xlabel('detuning, MHz') #in real frequencyes
    ylabel('atoms number, a.u.')
    ylim(bottom=0)
    #start, end = ax.get_xlim()
    #print(ax.get_xlim())
    #ax.xaxis.set_ticks(arange(start, end, 0.1))
    #axvline(x=l_pos)
    fg = fig
    
ter = interactive(plot_gr,l_pos=(150,250,0.1),**ch_boxes)
ter.box_style='info'
display(ter)

###For managing clock line shift and broadening

#### Построение списка сканирования

In [ ]:
res = array(range(-7,7, 1))/10 + 420.1
rr = ''
for x in res:
    rr += str(x) + ' '
print(rr)